# **Linear Algebra for Data Science**: HW3, task 5 (3 pts)
# Iterative methods for eigenvalue calculations

### <div align="right"> &copy; Yurii Yeliseev & Rostyslav Hryniv, 2023 </div>

## Completed by:   
*   Roman Kovalchuk
*   Eduard Pekach

The aim of this task is to implement two iterative methods for eigenvalue and eigenvector calculations, the Jacobi and QR ones.


# 1. Jacobi method **(1 pt)**
The Jacobi method is applicable to symmetric matrices $A$ and uses the ***Givens rotations*** (also called  ***Jacobi rotations*** in this context) that consecutively decrease the off-diagonal part $$\mbox{off}(A):=\sum_{i\ne j} |a_{ij}|^2$$ and thus transform $A$ to "nearly" diagonal form. (See the corresponding task from the first homework for details on the Givens rotations.) If $$A_k = Q_kQ_{k-1}\cdots Q_2Q_1 A Q_1^\top Q_2^\top \cdots Q_{k-1}^\top Q_k^\top$$ is alsmost diagonal, then its diagonal entries are close to the eigenvalues of $A$, and the approximation to eigenvectors can be obtained from the Givens rotations.

## 1.1. Algorithm justification
#### 1.1.1. Basic step
Assume that $A$ is an $n\times n$ symmetric matrix and that indices $p,q$ with $1\le p < q \le n$ are such that $a_{pq} \ne 0$. The task is to find an angle $\theta$ such that, with $c:=\cos\theta$ and $s:=\sin\theta$, the matrix
$$\begin{pmatrix} b_{pp} & b_{pq} \\ b_{qp} & b_{qq} \end{pmatrix} = \begin{pmatrix} c & -s \\ s & c \end{pmatrix}\begin{pmatrix} a_{pp} & a_{pq} \\ a_{qp} & a_{qq} \end{pmatrix}\begin{pmatrix}c & s \\ -s & c  \end{pmatrix}$$ is diagonal.


#### **Task 1.1.2**: Explain why $|b_{pp}|^2 + |b_{qq}|^2 > |a_{pp}|^2 + |a_{qq}|^2$

____
Consider Frobenius norm, which is invariant under orthogonal transformations;
and the fact that rows and columns p and q of A are modified in B:
$$
\text{off}(B)^2 = ||B||_F^2 - \sum_{i=1}^{n} b_{ii}^2 \\
= ||A||_F^2 - \sum_{i!=p, q} b_{ii}^2 - (b_{pp}^2 + b_{qq}^2) \\
= ||A||_F^2 - \sum_{i!=p, q} a_{ii}^2 - (a_{pp}^2 + 2 * a_{pq}^2 + a_{qq}^2) \\
= ||A||_F^2 - \sum_{i=1}^{n} a_{ii}^2 - 2 * a_{pq}^2 \\
= \text{off}^2 - 2a_{pq}^2 \\
< \text{off}^2
$$
---


#### **Task 1.1.3**: Find the angle $\theta$ so that the above matrix $B$ is diagonal
---
In order for matrix B to be diagonal, the $s$ parameter has to equal to 0, and $c$ equal to 1:
Following these relationships from Symmetric Schur Decomposition:
$$
\frac{s}{c} = t \text{ ,   } s^2 + c^2 = 1 \\
\text{where, } t = -\tau + / - \sqrt{ 1 + \tau^2} = \tan{\theta} \\
\text{Henceforth, } s \text{ has to equal: } 0 \\
\text{And, } c \text{ has to either equal to } 1 \text{ or } -1 \text{ to fulfill the relationship: } s^2 + c^2 = 1 \\
\text{Therefore, considering: } \tan{\theta} = t => \tan{\theta} = 0 \text{ as } \frac{s}{c} = t; s = 0 => \theta = \pi * n \text{ for any real n }
$$

---  


### **Task 1.1.4**: write a function that for a given matrix $A$ and indices $p<q$ finds the angle $\theta$ in the Jacobi rotation $J$  


In [12]:
import numpy as np
def calc_rotation_angle(A, p, q):
    if A[p, p] == A[q, q]:
        return np.pi / 4 if A[p, q] != 0 else 0
    else:
        return np.arctan(2 * A[p, q] / (A[q, q] - A[p, p])) / 2

## 1.2 The algorithm and its implementation
Denote by $$J(p,q,\theta): = \begin{pmatrix}1 & \dots & 0 & \dots & 0 &  \dots & 0 \\ \vdots & \ddots & \vdots & & \vdots & & \vdots \\ 0 & \dots & c & \dots & s & \dots & 0 \\ \vdots & \ddots & \vdots & & \vdots & & \vdots \\ 0 & \dots & -s & \dots & c & \dots & 0 \\ \vdots & \ddots & \vdots & & \vdots & & \vdots \\ 0 & \dots & 0 & \dots & 0 &  \dots & 1 \end{pmatrix}$$ the Jacobi (Givens) rotation in entries $p<q$.

Given a symmetric $n\times n$ matrix $A$ and a positive
tolerance $\mbox{tol}$, we find an orthogonal $V = J_1 J_2\cdots J_k$ such that $B:= V^\top A V$ has small off-diagonal part, namely $\mbox{off}(B) \le \delta:= \mbox{tol}\cdot \|A\|_F$.

#### **Initialize:** $V = I_n$, $B=A$
#### **while** $\mbox{off}(B) > \delta$:  
- find $p<q$ s.t. $|b_{pq}| = \max_{i\ne j}|b_{ij}|$
- set $\theta = \texttt{rot_angle}(B, p, q)$
- update $B = J(p,q,\theta)^\top B J(p,q,\theta)$
- update $V = V J(p,q,\theta)$

#### **end**




---
### **Task 1.2.1**: Implement the above algorithm

In [13]:
def jacobi_method(A, tol=1e-8, max_iter=1000):
    """
    The jacobi_method function is used to calculate the eigenvalues and eigenvectors of a square matrix using the Jacobi method.

    Inputs:

    A (numpy.ndarray): The square matrix for which the eigenvalues and eigenvectors are to be calculated.
    tol (float): The tolerance level. The default value is 1e-8.
    max_iter (int): The maximum number of iterations to perform. The default value is 1000.
    Outputs:

    numpy.ndarray: The eigenvalues of the input matrix.
    numpy.ndarray: The eigenvectors of the input matrix.
    """
    n = A.shape[0]
    V = np.eye(n)
    B = np.copy(A)
    off_A = np.sqrt(np.sum(np.square(A)) - np.sum(np.square(np.diag(A))))
    delta = tol * off_A
    
    for e in range(max_iter):
        off_B = np.sqrt(np.sum(np.square(B)) - np.sum(np.square(np.diag(B))))
        if off_B <= delta:
            break
        
        max_val = 0
        p, q = 0, 1
        for i in range(n):
            for j in range(i+1, n):
                if abs(A[i, j]) > max_val:
                    max_val = abs(A[i, j])
                    p, q = i, j
                    
        if B[p, p] == B[q, q]:
            theta = np.pi / 4
        else:
            theta = 0.5 * np.arctan2(2 * B[p, q], B[q, q] - B[p, p])
        J = np.eye(len(B))
        J[p, p] = J[q, q] = np.cos(theta)
        J[p, q] = -np.sin(theta)
        J[q, p] = np.sin(theta)

        B = J.T @ B @ J
        V = V @ J

    eigenvalues = np.diag(B)
    eigenvectors = V
    return eigenvalues, eigenvectors

Observe that on each step the norm of the off-diagonal part of $B$ decreases at least by the factor $(N-1)/N$, where $N:=n(n-1)$. Therefore, the algorithm converges in a finite number of steps

---
### **Task 1.2.2**: Eigenvalues and eigenvectors by Jacobi method

##### Given that $B = V^\top AV$ is almost diagonal, identify the approximate eigenvalues and eigenvectors of $A$
---
When we come up with
$$
V^T A V = B;
$$
where $D$ is diagonal and $V$ is a rotation matrix. Then the diagonal entries of $B$ are the
eigenvalues of $A$ and the columns of $V$ are eigenvectors of $A$.

The off diagonal terms will tend to zero in the limit as the number of iterations goes to
infinity. In fact, we can guarantee a limit on the number of iterations it will take before
the off diagonal entries all round off to zero.


---

# 2. QR-method **(1 pt)**

The $QR$ eigenvalue method is based on the observation that, under mild assumptions on an $n\times n$ matrix $A$, the sequence constructed starting with $A_1 = A$ and the update step $$A_k = Q_kR_k \quad \mapsto \quad A_{k+1}:=R_kQ_k$$
coverges to an upper-triangular $A_*$ holding the eigenvalues of the initial matrix $A$


## 2.1 Justification of the method

The fact that the sequence $A_k$ converges to an upper-triangular limit is rather difficult to prove and is skipped here. We will justify that the method gives the eigenvalues of $A$ and, in symmetric case, also the corresponding eigenvectors

### 2.1.1. Eigenvalues
Explain why the diagonal entries of the upper-triangular limiting matrix $A_* = \lim_{k\to\infty} A_k$ are eigenvalues of $A$ *(Hint: show that all matrices $A_k$ are similar to $A$)*

---
The basic QR algorithm can be visualized in the case where A is a positive-definite symmetric matrix. In that case, A can be depicted as an ellipse in 2 dimensions or an ellipsoid in higher dimensions.

It's essential to note that computationally finding a single eigenvector of a symmetric matrix is generally not feasible, especially when the multiplicities of the eigenvalues are unknown. However, the computational complexity issue doesn't apply to finding eigenvalues, as they are always computable.

In the context of the basic QR algorithm, consider the case where \(A\) is a positive-definite symmetric matrix, visualized as an ellipse in 2D or an ellipsoid in higher dimensions. As the two eigenvalues of the input matrix approach each other, the ellipse transforms into a circle, corresponding to a multiple of the identity matrix. The near-circle represents a near-multiple of the identity matrix with eigenvalues nearly equal to the diagonal entries of the matrix. This illustrates that the problem of approximately finding eigenvalues is manageable in this scenario. However, the challenge lies in determining eigenvectors, which can only be accurately known when the semi-axes of the ellipses are parallel to the coordinate axes. As the input ellipse becomes more circular, the number of iterations required to achieve near-parallelism increases without bound.

---

### 2.1.2. Eigenvectors
Assume now that the initial matrix $A$ is symmetric. Explain why all matrices $A_k$ are also symmetric. Conclude that the limiting matrix $A_*$ is diagonal. Then identify an orthogonal matrix $V$ such that $A_* = V^\top AV$ and thus find the corresponding eigenvectors of $A$

---
### 2.1.2. Eigenvectors

When the initial symmetric matrix $A$ undergoes the QR algorithm, all subsequent matrices $A_k$ in the iteration remain symmetric due to the orthogonal transformations involved, keeping the symmetry.

The symmetry of the matrix $A_*$ is a consequence of the symmetry of each $A_k$ matrix. As $A_*$ represents the limit of the sequence of symmetric matrices $A_k$, it also has the symmetry property, one of the characteristics of the QR algorithm.

In the context of the spectral decomposition of a symmetric matrix $A = VDV^\top$, where $V$ is an orthogonal matrix of eigenvectors and $D$ is a diagonal matrix of eigenvalues, the limiting matrix $A_*$ can be expressed as

$$

A_* = V_*D_*V_*^\top

$$
Where, $V_*$ is orthogonal, and $D_*$ is diagonal.

By identifying $V$ as $V_*$, we can express $A_* = V^\top AV$, illustrating that the matrix $V$ has the eigenvectors of $A$, diagonalizing $A_*$ with $V$.

---

## 2.2. Implementation of the method
No need to implement the QR factorization from the scratch (although you may use your solutions of the HW1 based on the Givens rotations and/or Householder reflections)

In [14]:
def QR_method(A, tol=1e-10, max_iter=1000):
    """The QR_method function is used to calculate the eigenvalues of a square matrix using the QR method.

    Inputs:

    A (numpy.ndarray): The square matrix for which the eigenvalues are to be calculated.
    tol (float): The tolerance level. The default value is 1e-10.
    max_iter (int): The maximum number of iterations to perform. The default value is 1000.
    Outputs:

    numpy.ndarray: The diagonal of the final transformed matrix A, which represents the eigenvalues of the input matrix"""
    n = A.shape[0]
    Q_total = np.eye(n)

    for _ in range(max_iter):
        Q, R = np.linalg.qr(A)
        A = R @ Q
        Q_total = Q_total @ Q

        # Check for convergence
        off_diagonal = A - np.diag(np.diagonal(A))
        if np.all(np.abs(off_diagonal) < tol):
            break

    eigenvalues = np.diag(A)
    eigenvectors = Q_total
    return eigenvalues, eigenvectors

# 3. Testing **(0.5 pts)**

## 3.1 Initializing $A$
Generate a $10\times 10$ matrix $A = QDQ^{\top}$, where $Q$ is a (randomly generated) orthogonal matrix  and $D=\mbox{diag}\{1,2,\dots,10\}$ is a diagonal matrix. To generate $Q$, start with a $10\times 10$ matrix $B$ with random entries in $(0,1)$ and apply the $QR$-factorisation of $B$, $B=QR$, to determine $Q$. Alternatively, apply Gram-Schmidt to find an orthonormal basis of $\mathbb{R}^{10}$

In [15]:
B = np.random.rand(10, 10)
Q, R = np.linalg.qr(B)
D = np.diag(range(1, 11))
A = Q @ D @ Q.T

## 3.2 Eigenvalue and eigenvector calculations

### 3.2.1. Jacobi method
Apply Jacobi method to determine the eigenvalues and eigenvectors

In [18]:
eigenvalues_jac, eigenvectors_jac = jacobi_method(A)
eigenvalues_jac, eigenvectors_jac

(array([6.08479562, 6.11995737, 5.51617012, 4.92232377, 7.38782061,
        4.78346683, 3.65259682, 7.12359033, 3.4625272 , 5.94675134]),
 array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.81005847,
          0.        ,  0.5863491 ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.     

### 3.2.2 QR method
Apply the QR method to determine the eigenvalues and eigenvectors

In [19]:
eigenvalues_qr, eigenvectors_qr = QR_method(A)
eigenvalues_qr, eigenvectors_qr

(array([10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.]),
 array([[-0.38827504, -0.10812079, -0.46393328, -0.26824613,  0.42298503,
         -0.29098992,  0.35594055, -0.12516819, -0.22668337,  0.3049996 ],
        [-0.32989625, -0.37221769,  0.52265921, -0.22670563, -0.09148574,
          0.13850316,  0.07923566, -0.55413498,  0.24490317,  0.1648641 ],
        [ 0.18678413, -0.03196293,  0.09776713, -0.46128275, -0.46893745,
         -0.6379503 , -0.15293978,  0.16247202, -0.00164273,  0.255101  ],
        [ 0.35739524,  0.20960899, -0.06526827, -0.36284015,  0.18445513,
          0.38349047, -0.43174123, -0.29894565, -0.32434546,  0.36105705],
        [-0.15269373,  0.46229574,  0.47521722,  0.04014784, -0.05319852,
          0.15719125,  0.41836337,  0.32229099, -0.26381233,  0.39936012],
        [-0.07454401, -0.35955736,  0.08081616,  0.55131404, -0.17266948,
         -0.16811356, -0.21209908, -0.13008467, -0.63156729,  0.18926456],
        [ 0.26713997, -0.31242462, -0.3967106 

## 3.3 Evaluating the results

Compare the results with the built-in function and comment on the outcomes

In [22]:
eigenvalues_bi, eigenvectors_bi = np.linalg.eig(A)
eigenvalues_bi, eigenvectors_bi

(array([10.,  1.,  9.,  8.,  2.,  7.,  6.,  5.,  3.,  4.]),
 array([[ 0.38827504, -0.3049996 ,  0.10812079, -0.46393328, -0.22668337,
         -0.26824613, -0.42298503,  0.29098992,  0.12516819,  0.35594055],
        [ 0.32989625, -0.1648641 ,  0.37221769,  0.52265921,  0.24490317,
         -0.22670563,  0.09148574, -0.13850316,  0.55413498,  0.07923566],
        [-0.18678413, -0.255101  ,  0.03196293,  0.09776713, -0.00164273,
         -0.46128275,  0.46893745,  0.6379503 , -0.16247202, -0.15293978],
        [-0.35739524, -0.36105705, -0.20960899, -0.06526827, -0.32434546,
         -0.36284015, -0.18445513, -0.38349047,  0.29894565, -0.43174123],
        [ 0.15269373, -0.39936012, -0.46229574,  0.47521722, -0.26381233,
          0.04014784,  0.05319852, -0.15719125, -0.32229099,  0.41836337],
        [ 0.07454401, -0.18926456,  0.35955736,  0.08081616, -0.63156729,
          0.55131404,  0.17266948,  0.16811356,  0.13008467, -0.21209908],
        [-0.26713997, -0.3847075 ,  0.31242462

In [26]:
np.linalg.norm(eigenvectors_bi - eigenvectors_jac), np.linalg.norm(eigenvectors_bi - eigenvectors_qr), np.linalg.norm(eigenvectors_qr - eigenvectors_jac)

(4.329578362386122, 4.690415759841724, 4.458390713590455)

---
Above is the comparison with Frobenius norm differences:
1. Build In Numpy (Braman-Byers-Mathias Hessenberg QR) vs Jacobi Method
2. Build In Numpy (Braman-Byers-Mathias Hessenberg QR) vs QR Method
3. QR Method vs Jacobi.

Worth noting, that Numpy uses Braman-Byers-Mathias Hessenberg QR algorithm as a method to find eigenvalues and eigenvectors.
We can see that the jacobi method comes the closest to the numpy solution, however it does not imply that it gives the best (true) values everywhere.

The QR algorithm seems to be most of from its modification (Braman-Byers-Mathias Hessenberg) in build in numpy, most probably, due to severe accent in numpy
and this algorithm on additional iterations for more accuracy. However, this algorithm is well known for its longevity, when the eigenvalues are too close, which
increases number of iterations and slow-downs the process.

---


# 4. Conclusions **(0.5 pts)**
Summarize in several sentences what you achieved in this task, what were the main obstacles and how you overcome them. You can also add some suggestions how this task can be improved in the future

---
Jacobi:
In general, the Jacobi algorithm does not produce an exactly diagonal matrix in any
finite number of iterations, so the formulation of the algorithm that we gave above would
result in an infinite loop. However, it will very quickly “almost diagonalize” any matrix.

QR:
The QR algorithm's advantages include versatility, applicability to both real and complex eigenvalues, and effectiveness for symmetric matrices.
However, drawbacks include potentially slow convergence, particularly for matrices with clustered eigenvalues, and computational expense due to repeated orthogonal factorizations in each iteration.
Overall, the QR algorithm serves as a reliable numerical approach for eigenvalue computations, especially in situations where analytical solutions are challenging or unattainable. (Which is why probably the NumPy uses its variant)

Modifications:
However, the QR algorithm lacks numerical stability and efficiency which is dealt with using its modification (Braman-Byers-Mathias Hessenberg QR)

Summary:
Also in this lab, we have proven a few of underlying theorems, powering this numerical methods, which helped me to understand the relationships and how they work under the hood.
For example, the task when we had to find the theta value which would make B matrix diagonal, which requires understanding of different relationships (s, t and c) variables.

---